In [1]:
import datetime
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Download the data
def load_data() -> Tuple[Dataset, Dataset, Dataset]:
    from torchvision import datasets
    from torchvision.transforms import ToTensor

    training_data = datasets.FashionMNIST(
        root="data", train=True, download=True, transform=ToTensor()
    )

    test_data = datasets.FashionMNIST(
        root="data", train=False, download=True, transform=ToTensor()
    )

    validation_size = int(0.6 * len(test_data))
    test_size = len(test_data) - validation_size
    validation_data, test_data = random_split(
        test_data, [validation_size, test_size])
    return training_data, validation_data, test_data


training_data, validation_data, test_data = load_data()
training_loader = DataLoader(training_data, batch_size=4, shuffle=True)
# needs to be reproducible, so no shuffling
validation_loader = DataLoader(validation_data, batch_size=4, shuffle=False)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

# Model architecture:
```mermaid
flowchart 
    A{Input: \n2D 28x28 array} 
    A -->|28, 28| B[Flatten]
    B -->|784| C[Linear + ReLU]
    C -->|512| D[Linear + ReLU]
    D -->|512| E[Linear]
    E -->|10| F{Output: \nprobabilities of\neach label}
```


In [3]:
# Create the model
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")


class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


config = {}

model = Model(config).to(device)

Using cpu device


In [4]:
# Train the model
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


def train_one_epoch(tb_writer=None, epoch_index=0):
    running_loss = 0.0
    last_loss = 0.0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000  # loss per batch
            print("  batch {} loss: {}".format(i + 1, last_loss))
            if tb_writer is not None:
                tb_writer.add_scalar('Training loss', running_loss /
                                     1000, epoch_index * len(training_loader) + i)

            running_loss = 0.0

    return last_loss


start_timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f'runs/testing_{start_timestamp}')

EPOCHS = 5
best_validation_loss = 1e9

for epoch in range(EPOCHS, 2*EPOCHS):
    print(f"EPOCH {epoch + 1}")

    # Train the model
    model.train(True)
    avg_loss = train_one_epoch(writer, epoch)

    running_validation_loss = 0.0

    # Test it's intermediate performance
    model.eval()
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_validation_loss += vloss
    avg_vloss = running_validation_loss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    writer.add_scalars('Training vs. Validation loss', {
                       'Training': avg_loss, 'Validation': avg_vloss}, epoch + 1)
    writer.flush()

    if avg_vloss < best_validation_loss:
        best_validation_loss = avg_vloss
        model_path = f"model_{start_timestamp}_{epoch}"
        torch.save(model.state_dict(), model_path)

EPOCH 6


  batch 1000 loss: 1.3545449294298888


  batch 2000 loss: 0.7317184730656445


  batch 3000 loss: 0.6615200548283756


  batch 4000 loss: 0.5781671963580884


...

  batch 12000 loss: 0.2920735901299649


  batch 13000 loss: 0.29579464676836503


  batch 14000 loss: 0.29988778893547713


  batch 15000 loss: 0.3047588032895583


LOSS train 0.3047588032895583 valid 0.3930676579475403
